**<mark>Function to Run SQL Queries</mark>**

In [1]:
from   deltalake     import  write_deltalake,DeltaTable
import duckdb
import re
def run_sql(list_files):
    successful_runs = 0
    for x in list_files:
        try:
            file_path = f'{sql_folder}/{x}.sql'
            with open(file_path, 'r') as file:
                sql_content = file.read()

            match = re.search(r"-- materialized:\s*\((.*?)\)", sql_content)

            if match:
                materialized_content = match.group(1)
                parts = [part.strip() for part in materialized_content.split(',')]

                if len(parts) >= 3:
                    schema     = parts[0]
                    table_name = parts[1]
                    mode       = parts[2]
                    try:
                        write_delta(sql_content,table_name,schema, mode)
                        duckdb.sql(f""" create or replace view {table_name} as select * from delta_scan('{table_base_url}{schema}/{table_name}') """)
                        successful_runs = successful_runs +1
                    except:
                        print(f"Data not updated in {x}")
                else:
                    try:
                            duckdb.sql(sql_content)
                            print(f"Data updated in {x}")
                            successful_runs = successful_runs +1
                    except:
                            print(f"Data not updated in {x}")

        except FileNotFoundError:
            print(f"Error: The file '{file_path}' was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")      
    return successful_runs
def write_delta(sql_content,tbl,schema, mode):
            tbl_path = table_base_url + schema + '/' +tbl
            RG=8_000_000
            df = duckdb.sql(sql_content).arrow()
            r = df.num_rows
            write_deltalake( tbl_path,
            df,
            mode=mode,
            max_rows_per_file = RG ,
            max_rows_per_group = RG,
            min_rows_per_group = RG,
            engine ='pyarrow')
            if mode !='ignore':
                dt = DeltaTable(tbl_path)
                if len(dt.files()) > 90:
                    dt.optimize.compact()
                    dt.vacuum(retention_hours=0, dry_run=False, enforce_retention_duration=False)
                    dt.cleanup_metadata()
            return print(f'{r} rows inserted into {tbl}')


In [2]:
sql_folder     = './builtin/sql/'
table_base_url = 'abfss://processing@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Tables/'

In [3]:
run_sql(['scada_setup','price_setup','scada_today_setup','price_today_setup','duid_setup','summary_setup'])
run_sql(['scada_today','price_today'])
t = run_sql(['scada','price'])
print(f""" Nbr of tables changed {t}:  if any changes do backup otherwise do incremental """)
if t > 0 :
    run_sql(['calendar','mstdatetime','duid','summary_backfill'])
    run_sql(['summary_incremental'])
else :
    run_sql(['summary_incremental'])

0 rows inserted into scada
0 rows inserted into price
0 rows inserted into scada_today
0 rows inserted into price_today
0 rows inserted into duid
0 rows inserted into summary
279 rows inserted into scada_today


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

24490 rows inserted into price_today
Data not updated in scada
Data not updated in price
 Nbr of tables changed 0:  if any changes do backup otherwise do incremental 
276 rows inserted into summary
